In [1]:
import numpy as np 
import pandas as pd 
import xlrd
import re
from sklearn.feature_extraction.text import CountVectorizer

### English Word Test

In [2]:
with open("./Data Files/EN_US.txt") as word_file:
    english_words = set(word.strip().lower() for word in word_file)
def is_word(word):
    return word.lower() in english_words

### Methods to classify text

In [3]:
def extract_processed_words(series_): #series is the bank statement/narration
    df = pd.Series(series_)
    df.columns = ['narration']
    hyphen_text = df['test'].str.extract("^([A-Z]+-[A-Z]+[A-Z\s]*)\s",expand=True)
    upper_case_text = df['test'].str.extract("^([A-Z\s]*)\s",expand=True)
    forex.columns = ['Features']
    interest.columns = ['Features']
    hyphen_text = hyphen_text.dropna()
    hyphen_text['Features'] = hyphen_text['Features'].str.split('-| ').str.lower()
    upper_case_text = upper_case_text.dropna()
    upper_case_text['Features'] = upper_case_text['Features'].str.split(' ')
    combined = pd.concat([hyphen_text,upper_case_text]).sort_index().reset_index()
    df_length = combined['index'][len(combined)-1]
    combined = combined.set_index('index')
    new_index = pd.Index(np.arange(0,df_length+1,1))
    combined = combined.reindex(new_index).reset_index()
    combined = combined.drop(['index'],axis=1)
    return combined

In [4]:
def get_english_words_out_of_string(string):
    output_list = [string]
    ticker_list = []
    for splits in string.split(' '):
        if is_word(splits):
            output_list.append(splits.lower())
        else:
            break
    for splits in string.split(' '):
        if splits.lower() in output_list:
            pass
        else:
            ticker_list.append(splits)
    return output_list,' '.join(ticker_list)

In [5]:
print(get_english_words_out_of_string("Loan Payment 0546-00866537.90 0008"))

(['Loan Payment 0546-00866537.90 0008', 'loan', 'payment'], '0546-00866537.90 0008')


In [6]:
def get_vectorized_result(series_):
    nar_list = []
    output_list = []
    for i in series_:
        l = get_english_words_out_of_string(i)[0]
        l[1:] = [' '.join(l[1:])] 
        nar_list.append(l[0])
        output_list.append(l[1])      
    vectorizer = CountVectorizer(min_df=0)
    X = vectorizer.fit_transform(pd.Series(output_list))
    df = pd.DataFrame(X.toarray())
    df = df.set_index(pd.Series(nar_list))
    df.columns = vectorizer.get_feature_names()
    return df

In [7]:
def decision_tree(df):
    row,col = df.shape
    output_list = ['Manual'] * row
    df = df.reset_index(drop=True)
    distribution_i = np.sort(df.index[df['dividend'] == 1])
    purchase_i = np.sort(df.index[df['investment'] == 1])
    loanCreate_i = np.sort(df.index[df['payment'] == 1])
    loanRepay_i = np.sort(df.index[df['repayment'] == 1])
    loanCost_i = np.sort(df.index[df['interest'] == 1])
    for x in distribution_i:
        output_list[x] = 'Distribution'
    for x in purchase_i:
        output_list[x] = 'Purchase'
    for x in loanCreate_i:
        output_list[x] = 'LoanCreate'
    for x in loanRepay_i:
        output_list[x] = 'LoanRepay'
    for x in loanCost_i:
        output_list[x] = 'LoanCost'
    return output_list

In [23]:
def classifier(list_):
    if 'dividend' in list_:
        return 'Distribution'
    elif 'investment' in list_:
        return 'Purchase'
    elif 'payment' in list_:
        return 'LoanCreate'
    elif 'repayment' in list_:
        return 'LoanRepay'
    elif 'interest' in list_:
        return 'LoanCost'
    else:
        return r'Please do this manually'

In [28]:
def split_and_decide(series):
    output_list = []
    for i in series:
        l = get_english_words_out_of_string(i)[0] #list of splits
        transactionType = classifier(l)
        output_list.append(transactionType)
    return (output_list)

### Solver

In [25]:
excel_file = './Data Files/ubs_raw_statement.xlsx'

In [30]:
def UBS_solver(excel_file):
    df = pd.read_excel(excel_file)
    
    
    #easy rows
    #CcyAccountCode should have curency column
    output_col = ['Action','CcyAccountCode','TradeDate','ValueDate','Narration','Amount','TransactionType','Ticker','Quantity','Price','DirtyPrice','HashTags']
    output_df = pd.DataFrame(columns=output_col)
    output_df['TradeDate'] = df['Trade date']
    output_df['ValueDate'] = df['Value date']
    output_df['Narration'] = df['Information']
    output_df['Amount'] = -df['Debit']
    output_df['Amount'] = output_df['Amount'].add(df['Credit'],fill_value = 0)
    output_df.drop(output_df.index[output_df['Amount'].isna()],inplace=True)
    output_df.index = range(len(output_df))
    output_df['Action'] = pd.Series(['CreateTransaction']*len(output_df))
    
    #CcyAccountCode
    ccyaccountcode = 'Dummy Account Number'
    output_df['CcyAccountCode'] = pd.Series([ccyaccountcode]*len(output_df))
    
    #text classification
    #table_for_decision_tree = get_vectorized_result(output_df['Narration'])
    #classified_text_df = decision_tree(table_for_decision_tree)
    classified_text_df = split_and_decide(output_df['Narration'])
    output_df['TransactionType'] = pd.Series(classified_text_df)
    #ticker
    LoanRepay = list(output_df.index[output_df['TransactionType'] == 'LoanRepay'])
    LoanCost = list(output_df.index[output_df['TransactionType'] == 'LoanCost'])
    LoanCreate = list(output_df.index[output_df['TransactionType'] == 'LoanCreate'])
    Loan = LoanRepay + LoanCost + LoanCreate
    for i in Loan:
        output_df['Ticker'][i] = '_'.join(["LoanRef",output_df["CcyAccountCode"][i],get_english_words_out_of_string(output_df['Narration'][i])[1]])
    
    
    return output_df

In [31]:
writer = pd.ExcelWriter('./Data Files/ubs_statement_processed.xlsx')
UBS_solver(excel_file).to_excel(writer,'UBS_Processed',index=None)
writer.save()

C:\Users\Justin Leung\AppData\Local\conda\conda\envs\datafeed\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
